In [1]:
import time
import datetime
import math
import sys
import gurobipy
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
from utility import load_specific_yaml

print(f"python version: {sys.version}")
print(f"gurobipy version: {gurobipy.gurobi.version()}")


python version: 3.9.18 (main, Sep 11 2023, 08:38:23) 
[Clang 14.0.6 ]
gurobipy version: (11, 0, 0)


In [3]:
# Load experiments yaml
config = load_specific_yaml('instance_F_3.yaml')
# config

I = set(range(0, config['i_amount']))  
J = set(range(0, config['j_amount']))  
K = set(range(0, config['k_amount']))  
L = set(range(0, config['l_amount']))  

In [ ]:
'''
Decision Variables
===============================
y : Whether location j build facility or not, Dim: (j) , BINARY
x : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS
aX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS
===============================

Other Variables
u : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS
P : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS
TA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS
===============================
'''

# Model initial (G, no E, no reform)
model = Model("Competitive Facility Location")
model.setParam('NumericFocus', 3)
y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="utility(u)", lb=0) # utility var(3.5)
P = model.addVars(I, vtype=GRB.CONTINUOUS, name="P, utility vs TA ratio", lb=0.0) # Probability of i choosing j
TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000000)
TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="negative TAi*lambda(e's power)", lb= -100000, ub= -0.000001) #-lambda*TAi
G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars", lb=0.0, ub=1) #e^-(lambda*TAi), ub=0.5
T = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000000)

model.update()

model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

model.addConstrs((u[i, j] == ((quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j]) / (config['D'][i][j]**2)) for i in I for j in J), "(3.5) utility for facility j to customer i")
# model.addConstrs((u[i, j] == P[i, j] for i in I for j in J), "(3.8) calculate P by uij/TAi, 分母要是常數所以用乘的")
## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + (quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) for i in I), "(3.7) calculate total TA by u and A_bar")
model.addConstrs((T[i] * TA[i] <= 1 for i in I), 'Fractional t')
model.addConstrs((P[i] == (quicksum(u[i, j] for j in J) * T[i]) for i in I), 'F(X) / G(X)')

model.addConstrs((TAi_lambda[i] == (-config['lambda_for_G'] * TA[i]) for i in I), "negative tai*lambda constr")
# Gurobi無法直接在目標式有exponential
for i in I:
    model.addGenConstrExpA(TAi_lambda[i], G_exp_vars[i], math.e, name=f"G_exp_constr_{i}")



# Objective function: Maximize profit by attracting customers - costs
objective = ((quicksum(config['H'][i] * (1 - G_exp_vars[i]) * P[i] for i in I)) \
          - (quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J)))
model.setObjective(objective, GRB.MAXIMIZE)


# Solve and Output
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
else:
    print("No optimal solution found.")
    model.computeIIS()
    # 将IIS输出到文件
    # model.write("model_iis.mps")
    print("IIS written to file 'model_iis.mps'")

In [ ]:
for var in model.getVars():
    print(f"{var.varName} = {var.x}")
print("")

y[0] = 1.0
y[1] = 0.0
y[2] = 1.0
y[3] = 0.0
y[4] = 0.0
y[5] = 0.0
y[6] = 0.0
y[7] = 0.0
y[8] = 0.0
y[9] = 1.0
y[10] = 0.0
y[11] = 0.0
x[0,0] = 2.0
x[0,1] = 0.0
x[0,2] = 0.7505452795670196
x[1,0] = 0.0
x[1,1] = 0.0
x[1,2] = 0.0
x[2,0] = 2.3965386808638605
x[2,1] = 2.0
x[2,2] = 0.0
x[3,0] = 0.0
x[3,1] = 0.0
x[3,2] = 0.0
x[4,0] = 0.0
x[4,1] = 0.0
x[4,2] = 0.0
x[5,0] = 0.0
x[5,1] = 0.0
x[5,2] = 0.0
x[6,0] = 0.0
x[6,1] = 0.0
x[6,2] = 0.0
x[7,0] = 0.0
x[7,1] = 0.0
x[7,2] = 0.0
x[8,0] = 0.0
x[8,1] = 0.0
x[8,2] = 0.0
x[9,0] = 0.0
x[9,1] = 4.97059972821324
x[9,2] = 0.0
x[10,0] = 0.0
x[10,1] = 0.0
x[10,2] = 0.0
x[11,0] = 0.0
x[11,1] = 0.0
x[11,2] = 0.0
aX[0] = 0.0
aX[1] = 0.0
aX[2] = 0.0
aX[3] = 0.0
aX[4] = 0.0
aX[5] = 0.0
aX[6] = 0.0
aX[7] = 0.0
aX[8] = 0.0
aX[9] = 0.0
aX[10] = 0.0
aX[11] = 0.0
utility(u)[0,0] = 123663.06940308373
utility(u)[0,1] = 0.0
utility(u)[0,2] = 21947.87968290926
utility(u)[0,3] = 0.0
utility(u)[0,4] = 0.0
utility(u)[0,5] = 0.0
utility(u)[0,6] = 0.0
utility(u)[0,7] = 0.